In [1]:
import numpy as np 
import math
import random as rd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
def create_data(n_rows, n_cols,choice):
    x_sample=[]
    col=[]
    while len(x_sample)<n_rows:
        if choice:
            col=[rd.randint(-1000,1000) for _ in range(n_cols)]
        else:
            col=[rd.choice([0,1]) for _ in range(n_cols)]
        x_sample.append(col)
    x_sample= np.array(x_sample)
    return x_sample

x = create_data(1000, 42,True)
y= create_data(1000, 1,False)


In [2]:
x_train = x[:700]
x_test=x[700:]
y_train=y[:700]
y_test=y[700:]
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(700, 42) (700, 1) (300, 42) (300, 1)


In [3]:
from __future__ import print_function
from __future__ import absolute_import

import warnings
import numpy as np

from keras.layers import Input, Conv1D, BatchNormalization, Activation, Dropout, Reshape, GlobalAveragePooling1D,MaxPooling1D,Dense,AveragePooling1D
from keras.models import Model
from keras import layers

from keras.layers import Layer
from keras import backend as K


WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'


def conv1d_bn(x,
              filters,
              kernel_size,
              padding='same',
              strides=1,
              name=None):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 1
    x = Conv1D(
        filters, kernel_size,
        strides=strides,
        padding=str(padding),
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x


def InceptionV3():
    # Determine proper input shape
    input_shape = (42,1)
    x_input = Input(shape=input_shape)
    channel_axis = -1

    x = conv1d_bn(x_input, 32, 3, strides=2, padding='valid')
    x = conv1d_bn(x, 32, 3, padding='valid')
    x = conv1d_bn(x, 64, 3, padding='valid')
    x = MaxPooling1D(3, strides=2, name="block1_pool")(x)
    # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv1d_bn(x, 64,1,padding='same')

    branch5x5 = conv1d_bn(x, 48,1,padding='same')
    branch5x5 = conv1d_bn(branch5x5, 64, 5,padding='same')
    branch3x3dbl = conv1d_bn(x, 64, 1,padding='same')
    branch3x3dbl = conv1d_bn(branch3x3dbl, 96, 3,padding='same')
    branch3x3dbl = conv1d_bn(branch3x3dbl, 96, 3,padding='same')
    branch_pool = AveragePooling1D(3, strides=1, padding='same')(x)
    branch_pool = conv1d_bn(branch_pool, 32, 1,padding='same')
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')

    # mixed 1: 35 x 35 x 256
    branch1x1 = conv1d_bn(x, 64,1,padding='same')
    branch5x5 = conv1d_bn(x, 48, 1,padding='same')
    branch5x5 = conv1d_bn(branch5x5, 64,5,padding='same')

    branch3x3dbl = conv1d_bn(x, 64, 1,padding='same')
    branch3x3dbl = conv1d_bn(branch3x3dbl, 96, 3,padding='same')
    branch3x3dbl = conv1d_bn(branch3x3dbl, 96, 3,padding='same')

    branch_pool = AveragePooling1D(3, strides=1, padding='same')(x)
    branch_pool = conv1d_bn(branch_pool, 64, 1,padding='same')
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')

    # mixed 2: 35 x 35 x 256
    branch1x1 = conv1d_bn(x, 64, 1,padding='same')

    branch5x5 = conv1d_bn(x, 48, 1,padding='same')
    branch5x5 = conv1d_bn(branch5x5, 64, 5,padding='same')

    branch3x3dbl = conv1d_bn(x, 64, 1,padding='same')
    branch3x3dbl = conv1d_bn(branch3x3dbl, 96, 3,padding='same')
    branch3x3dbl = conv1d_bn(branch3x3dbl, 96, 3,padding='same')

    branch_pool = AveragePooling1D(3, strides=1, padding='same')(x)
    branch_pool = conv1d_bn(branch_pool, 64, 1,padding='same')
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')

    # mixed 3: 17 x 17 x 768
    branch3x3 = conv1d_bn(x, 384, 3, strides=2, padding='valid')

    branch3x3dbl = conv1d_bn(x, 64, 1,padding='same')
    branch3x3dbl = conv1d_bn(branch3x3dbl, 96, 3,padding='same')
    branch3x3dbl = conv1d_bn(
        branch3x3dbl, 96, 3, strides=2, padding='valid')

    branch_pool = MaxPooling1D(3, strides=2)(x)
    x = layers.concatenate(
        [branch3x3, branch3x3dbl, branch_pool], axis=channel_axis, name='mixed3')

    # mixed 4: 17 x 17 x 768
    branch1x1 = conv1d_bn(x, 192, 1,padding='same')

    branch7x7 = conv1d_bn(x, 128, 1,padding='same')
    branch7x7 = conv1d_bn(branch7x7, 128, 1,padding='same')
    branch7x7 = conv1d_bn(branch7x7, 192, 7,padding='same')

    branch7x7dbl = conv1d_bn(x, 128, 1,padding='same')
    branch7x7dbl = conv1d_bn(branch7x7dbl, 128, 7,padding='same')
    branch7x7dbl = conv1d_bn(branch7x7dbl, 128, 1,padding='same')
    branch7x7dbl = conv1d_bn(branch7x7dbl, 128, 7,padding='same')
    branch7x7dbl = conv1d_bn(branch7x7dbl, 192, 1,padding='same')

    branch_pool = AveragePooling1D(3, strides=1, padding='same')(x)
    branch_pool = conv1d_bn(branch_pool, 192, 1,padding='same')
    x = layers.concatenate(
        [branch1x1, branch7x7, branch7x7dbl, branch_pool],
        axis=channel_axis,
        name='mixed4')

    # mixed 5, 6: 17 x 17 x 768
    for i in range(2):
        branch1x1 = conv1d_bn(x, 192, 1,padding='same')

        branch7x7 = conv1d_bn(x, 160, 1,padding='same')
        branch7x7 = conv1d_bn(branch7x7, 160, 1,padding='same')
        branch7x7 = conv1d_bn(branch7x7, 192, 7,padding='same')

        branch7x7dbl = conv1d_bn(x, 160, 1,padding='same')
        branch7x7dbl = conv1d_bn(branch7x7dbl, 160, 7,padding='same')
        branch7x7dbl = conv1d_bn(branch7x7dbl, 160, 1,padding='same')
        branch7x7dbl = conv1d_bn(branch7x7dbl, 160, 7,padding='same')
        branch7x7dbl = conv1d_bn(branch7x7dbl, 192, 1,padding='same')

        branch_pool = AveragePooling1D(
            3, strides=1, padding='same')(x)
        branch_pool = conv1d_bn(branch_pool, 192, 1,padding='same')
        x = layers.concatenate(
            [branch1x1, branch7x7, branch7x7dbl, branch_pool],
            axis=channel_axis,
            name='mixed' + str(5 + i))

    # mixed 7: 17 x 17 x 768
    branch1x1 = conv1d_bn(x, 192, 1,padding='same')

    branch7x7 = conv1d_bn(x, 192, 1,padding='same')
    branch7x7 = conv1d_bn(branch7x7, 192, 1,padding='same')
    branch7x7 = conv1d_bn(branch7x7, 192, 7,padding='same')

    branch7x7dbl = conv1d_bn(x, 192, 1,padding='same')
    branch7x7dbl = conv1d_bn(branch7x7dbl, 192, 7,padding='same')
    branch7x7dbl = conv1d_bn(branch7x7dbl, 192, 1,padding='same')
    branch7x7dbl = conv1d_bn(branch7x7dbl, 192, 7,padding='same')
    branch7x7dbl = conv1d_bn(branch7x7dbl, 192, 1,padding='same')

    branch_pool = AveragePooling1D(3, strides=1, padding='same')(x)
    branch_pool = conv1d_bn(branch_pool, 192, 1,padding='same')
    x = layers.concatenate(
        [branch1x1, branch7x7, branch7x7dbl, branch_pool],
        axis=channel_axis,
        name='mixed7')

    # mixed 8: 8 x 8 x 1280
    branch3x3 = conv1d_bn(x, 192, 1,padding='same')
    branch3x3 = conv1d_bn(branch3x3, 320, 3,
                          strides=2, padding='valid')

    branch7x7x3 = conv1d_bn(x, 192, 1,padding='same')
    branch7x7x3 = conv1d_bn(branch7x7x3, 192, 1,padding='same')
    branch7x7x3 = conv1d_bn(branch7x7x3, 192, 7,padding='same')
    branch7x7x3 = conv1d_bn(
        branch7x7x3, 192, 3, strides=2, padding='valid')

    branch_pool = MaxPooling1D(3, strides=2)(x)
    x = layers.concatenate(
        [branch3x3, branch7x7x3, branch_pool], axis=channel_axis, name='mixed8')

    # mixed 9: 8 x 8 x 2048
    for i in range(2):
        branch1x1 = conv1d_bn(x, 320, 1,padding='same')

        branch3x3 = conv1d_bn(x, 384, 1,padding='same')
        branch3x3_1 = conv1d_bn(branch3x3, 384, 1,padding='same')
        branch3x3_2 = conv1d_bn(branch3x3, 384, 3,padding='same')
        branch3x3 = layers.concatenate(
            [branch3x3_1, branch3x3_2], axis=channel_axis, name='mixed9_' + str(i))

        branch3x3dbl = conv1d_bn(x, 448, 1,padding='same')
        branch3x3dbl = conv1d_bn(branch3x3dbl, 384, 3,padding='same')
        branch3x3dbl_1 = conv1d_bn(branch3x3dbl, 384, 1,padding='same')
        branch3x3dbl_2 = conv1d_bn(branch3x3dbl, 384, 3,padding='same')
        branch3x3dbl = layers.concatenate(
            [branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

        branch_pool = AveragePooling1D(
            3, strides=1, padding='same')(x)
        branch_pool = conv1d_bn(branch_pool, 192, 1,padding='same')
        x = layers.concatenate(
            [branch1x1, branch3x3, branch3x3dbl, branch_pool],
            axis=channel_axis,
            name='mixed' + str(9 + i))
    x = GlobalAveragePooling1D(name='avg_pool')(x)
    x = Dense(1, activation='sigmoid', name='predictions')(x)

    # Create model.
    model = Model(x_input, x, name='inception_v3')
    return model


def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

model = InceptionV3()
lr=0.0001
model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 42, 1)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 20, 32)       96          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 20, 32)      60          ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 20, 32)       0           ['batch_normalization[

                                                                  'activation_9[0][0]']           
                                                                                                  
 conv1d_13 (Conv1D)             (None, 7, 64)        16384       ['mixed0[0][0]']                 
                                                                                                  
 batch_normalization_13 (BatchN  (None, 7, 64)       21          ['conv1d_13[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_13 (Activation)     (None, 7, 64)        0           ['batch_normalization_13[0][0]'] 
                                                                                                  
 conv1d_11 (Conv1D)             (None, 7, 48)        12288       ['mixed0[0][0]']                 
          

 conv1d_17 (Conv1D)             (None, 7, 64)        18432       ['mixed1[0][0]']                 
                                                                                                  
 conv1d_19 (Conv1D)             (None, 7, 64)        15360       ['activation_18[0][0]']          
                                                                                                  
 conv1d_22 (Conv1D)             (None, 7, 96)        27648       ['activation_21[0][0]']          
                                                                                                  
 conv1d_23 (Conv1D)             (None, 7, 64)        18432       ['average_pooling1d_2[0][0]']    
                                                                                                  
 batch_normalization_17 (BatchN  (None, 7, 64)       21          ['conv1d_17[0][0]']              
 ormalization)                                                                                    
          

 conv1d_34 (Conv1D)             (None, 3, 128)       16384       ['activation_33[0][0]']          
                                                                                                  
 batch_normalization_29 (BatchN  (None, 3, 128)      9           ['conv1d_29[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_34 (BatchN  (None, 3, 128)      9           ['conv1d_34[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_29 (Activation)     (None, 3, 128)       0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 activatio

 batch_normalization_44 (BatchN  (None, 3, 160)      9           ['conv1d_44[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_39 (Activation)     (None, 3, 160)       0           ['batch_normalization_39[0][0]'] 
                                                                                                  
 activation_44 (Activation)     (None, 3, 160)       0           ['batch_normalization_44[0][0]'] 
                                                                                                  
 conv1d_40 (Conv1D)             (None, 3, 160)       25600       ['activation_39[0][0]']          
                                                                                                  
 conv1d_45 (Conv1D)             (None, 3, 160)       179200      ['activation_44[0][0]']          
          

...

                                                                                                  
 batch_normalization_60 (BatchN  (None, 3, 192)      9           ['conv1d_60[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_65 (BatchN  (None, 3, 192)      9           ['conv1d_65[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_60 (Activation)     (None, 3, 192)       0           ['batch_normalization_60[0][0]'] 
                                                                                                  
 activation_65 (Activation)     (None, 3, 192)       0           ['batch_normalization_65[0][0]'] 
          

 ormalization)                                                                                    
                                                                                                  
 activation_69 (Activation)     (None, 1, 320)       0           ['batch_normalization_69[0][0]'] 
                                                                                                  
 activation_73 (Activation)     (None, 1, 192)       0           ['batch_normalization_73[0][0]'] 
                                                                                                  
 max_pooling1d_1 (MaxPooling1D)  (None, 1, 768)      0           ['mixed7[0][0]']                 
                                                                                                  
 mixed8 (Concatenate)           (None, 1, 1280)      0           ['activation_69[0][0]',          
                                                                  'activation_73[0][0]',          
          

C:\Users\PC\miniconda3\envs\test\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 ormalization)                                                                                    
                                                                                                  
 activation_76 (Activation)     (None, 1, 384)       0           ['batch_normalization_76[0][0]'] 
                                                                                                  
 activation_77 (Activation)     (None, 1, 384)       0           ['batch_normalization_77[0][0]'] 
                                                                                                  
 activation_80 (Activation)     (None, 1, 384)       0           ['batch_normalization_80[0][0]'] 
                                                                                                  
 activation_81 (Activation)     (None, 1, 384)       0           ['batch_normalization_81[0][0]'] 
                                                                                                  
 batch_nor

 activation_89 (Activation)     (None, 1, 384)       0           ['batch_normalization_89[0][0]'] 
                                                                                                  
 activation_90 (Activation)     (None, 1, 384)       0           ['batch_normalization_90[0][0]'] 
                                                                                                  
 batch_normalization_91 (BatchN  (None, 1, 192)      3           ['conv1d_91[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_83 (Activation)     (None, 1, 320)       0           ['batch_normalization_83[0][0]'] 
                                                                                                  
 mixed9_1 (Concatenate)         (None, 1, 768)       0           ['activation_85[0][0]',          
          

In [4]:
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 30)

early_stopping = EarlyStopping(monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=30,
    min_lr=0.000001,
    verbose=1,
)
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix

class ConfusionMatrixCallback(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data
        self.relist={}
        self.acc=[]

    def on_epoch_end(self, epoch, logs=None):
        X_val, y_val = self.validation_data
        y_pred = self.model.predict(X_val)
        y_pred_binary = (y_pred > 0.5).astype(int)

        cm = confusion_matrix(y_val, y_pred_binary)
        print(f"Confusion Matrix after Epoch {epoch + 1}:\n{cm}")
        print(classification_report(y_test,y_pred_binary))
        acc = float(classification_report(y_test,y_pred_binary).split('\n')[3].split()[2])
        self.acc.append(acc)
        self.relist[acc]=y_pred_binary

# ... Định nghĩa mô hình ...

# Tạo đối tượng callback
confusion_matrix_callback = ConfusionMatrixCallback(validation_data=(x_test,y_test))

callbacks = [early_stopping,lr_scheduler]

checkpointer = ModelCheckpoint(filepath = "Emotion_weights.hdf5", verbose = 1, save_best_only=True)

history = model.fit(x=x_train, y=y_train)



22/22 [==============================] - 26s 283ms/step - loss: 0.8033 - accuracy: 0.5114
